In [1]:
from typing import List

from tqdm import tqdm
import numpy as np
from nilearn.datasets import fetch_neurovault
from nibabel import Nifti1Image
from nilearn import plotting
from torchio import transforms
import torchio as tio

from ai4sipmbda.utils import fetching, difumo_utils

In [2]:
data = fetching.fetch_nv("/storage/store2/data/", max_images=10)

10 images found on local disk.


In [3]:
import sklearn
import pandas as pd

In [4]:
def get_dataset_labels(
    data: sklearn.utils.Bunch,
    study: str = "hcp"
) -> pd.DataFrame:
    """Fetches the relavant metadata from json files contained in base_path and
    creates labels in format compatible with the training pipeline

    Parameters
    ----------
    data : sklearn.utils.Bunch
        
    study : str, optional
        by default "hcp"

    Returns
    -------
    pandas.DataFrame
        DataFrame containing the subjects and contrast as used in the training
        scripts.
    """

    Y = list()
    for metadata in data["images_meta"]:
        Y.append({
            "study": study,
            "subject": metadata["name"].split("_")[0],
            "contrast": metadata["contrast_definition"],
            #"meta_path": os.path.join(data_dir, fname),
            # "path": os.path.join(data_dir, img_filename),
        })
    return pd.DataFrame(Y)

In [5]:
labels = get_dataset_labels(data)

In [6]:
AUGMENTATIONS = {
        "RandomElasticDeformation": tio.RandomElasticDeformation(num_control_points = 16, max_displacement = 2),
        "RandomMotion":tio.RandomMotion(degrees = 0.2, translation = 0.2, num_transforms = 2),
        "RandomGhosting": tio.RandomGhosting(num_ghosts = 1, intensity = 0.02, restore = 1.0),
        "RandomSpike": tio.RandomSpike(num_spikes = 2, intensity = 1.15),
        "RandomBiasField": tio.RandomBiasField(order = 1,coefficients=0.05 ),
        "RandomBlur": tio.RandomBlur(std = 1.05),
        "RandomNoise":tio.RandomNoise(mean = 0.3, std = 0.5),
        "RandomGamma": tio.RandomGamma(log_gamma=0.075),
        "RandomFlip": tio.RandomFlip(flip_probability=1.0),
    }

In [7]:
def create_augmentation(
    aug_names: List[str],
) -> transforms.Transform:
    augmentation_list = [AUGMENTATIONS[aug] for aug in aug_names]
    return tio.transforms.OneOf(augmentation_list)

In [8]:
flip = create_augmentation(["RandomFlip"])

In [9]:
Z_inv = np.load("hcp900_difumo_matrices/Zinv.npy")
mask = np.load("hcp900_difumo_matrices/mask.npy")

In [10]:
from joblib import Parallel, delayed

In [11]:
for task in labels.values:
    print(task)

['hcp' '106521' '0BK_PLACE']
['hcp' '117324' 'RF']
['hcp' '107321' 'SHAPES']
['hcp' '165638' '0BK_FACE']
['hcp' '465852' '0BK_TOOL']
['hcp' '124624' '0BK_FACE']
['hcp' '119126' 'RANDOM']
['hcp' '208226' 'RH']
['hcp' '395756' 'TOM']
['hcp' '105923' '0BK_TOOL']


In [24]:
def transform_based_augmentation(augmentation, mask, Z_inv, images_paths, labels, nb_fakes=10, n_jobs=1, verbose=0):

    def _create_fakes(image_path, task):
        print(f"Starting to augment {image_path}")

        image_tio = tio.ScalarImage(image_path)

        sub_X = [Z_inv.dot(image_tio.data.squeeze()[mask])]

        for _ in range(nb_fakes):
            # transform
            trf_img_tio = augmentation(image_tio)

            # project
            trf_difumo_vec = Z_inv.dot(trf_img_tio.data.squeeze()[mask])

            sub_X.append(trf_difumo_vec)
        print(f"Finished to augment {image_path}")
        return np.vstack(sub_X), np.vstack([task] * (1 + nb_fakes))

    
    parallel = Parallel(n_jobs=n_jobs, verbose=verbose)
    ret = parallel(delayed(_create_fakes)(image_path, task) for image_path, task in zip(images_paths, labels))

    X, Y = zip(*ret)

    return np.vstack(X), pd.DataFrame(np.vstack(Y), columns=labels.columns)

In [42]:
augmented_X, Y = transform_based_augmentation(flip, mask, Z_inv, data["images"], labels=labels, nb_fakes=2, n_jobs=3, verbose=10)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.5s finished


In [43]:
augmented_X.shape

(9, 1024)

In [13]:
all_augs = create_augmentation(list(AUGMENTATIONS.keys()))

In [21]:
augmented_X = transform_based_augmentation(all_augs, mask, Z_inv, data["images"], Y=None, nb_fakes=10, n_jobs=3, verbose=10)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    7.2s remaining:    0.0s
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    7.2s finished


In [22]:
augmented_X.shape

(30, 1024)

In [14]:
from joblib.parallel import Parallel, delayed
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ShuffleSplit

In [15]:
def preprocess_label(Y_t, use_dict=None, return_dict=False):
    """
    Preprocess label so that they match classifier like format

    Parameters
    ----------
    Y_t: pandas DataFrame
        labels to preprocess
    use_dict: dictionary
        If a dictionary labels -> class number is already known
    return_dict: bool
        If true, returns the dictionary labels -> class number
    Returns
    --------
    Y: np array
        np array where each label is replaced by its class number
    dict (optional): dict
        dictionary label -> class number (only returned if return_dict is True)
    """
    if use_dict is None:
        Y_dict = {v: k for k, v in enumerate(Y_t["contrast"].unique())}
    else:
        Y_dict = use_dict

    if return_dict:
        return Y_t["contrast"].apply(lambda x: Y_dict[x]).values, Y_dict
    else:
        return Y_t["contrast"].apply(lambda x: Y_dict[x]).values

In [16]:
class AugmentedClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model, f):
        """
        Trains a classifier using an augmentation method.
        Parameters
        ----------
        model: BaseEstimator
            The classifier used
        f: function (X, Y) -> X_fake, Y_fake
            The data augmentation function that
            generates fake (labeled) data from
            input data
        """

        self.model = model
        self.f = f

    def fit(self, X, Y):
        if self.f is None:
            return self.model.fit(X, Y)
        else:
            X_fake, Y_fake = self.f(X, Y)
            return self.model.fit(
                np.row_stack([X_fake, X]), np.concatenate([Y_fake, Y])
            )

    def predict(self, X, y):
        return self.model.predict(X, y)

    def score(self, X, y):
        return self.model.score(X, y)

In [21]:

def do_classif(
    images_path, labels, f, method_name, filename, train_size, n_splits=5, n_jobs=5
):
    """
    Tries 4 different classifier with the given augmentation method
    Parameters
    ----------
    X: np array of shape (n_samples, n_features)
        Input data
    Y: np array of shape (n_samples,)
        Labels
    f: function of X, Y
        f returns fake data and fake labels
    method_name: str
        the name of the method used to produce fake data
    filename: str
        filename is the path result file
    train_size : float or int, default=None
        If float, should be between 0.0 and 1.0 and represent the
        proportion of the dataset to include in the train split. If
        int, represents the absolute number of train samples. If None,
        the value is automatically set to the complement of the test size.
    n_splits : int, default=10
        Number of re-shuffling & splitting iterations.
    n_jobs: int, default: None
        The maximum number of concurrently running jobs,
    """

    models = []
    # parameters for MLP only
    params_2L = {
        "activation": "relu",
        "solver": "adam",
        "learning_rate": "constant",
        "momentum": 0.9,
        "learning_rate_init": 0.0001,
        "alpha": 0.00001,
        "random_state": 0,
        "batch_size": 32,
        "hidden_layer_sizes": (1024, 1024),
        "max_iter": 20000,
    }

    models.append(
        (LinearDiscriminantAnalysis(solver="lsqr", shrinkage="auto"), "LDA")
    )
    models.append((RandomForestClassifier(verbose=True), "RF"))
    # models.append((MLPClassifier(verbose=True, **params_2L), "MLP"))
    # models.append(
    #     (
    #         GridSearchCV(
    #             LogisticRegression(
    #                 solver="lbfgs",
    #                 tol=1e-4,
    #                 random_state=11,
    #                 penalty="l2",
    #                 max_iter=20000,
    #                 n_jobs=1,
    #                 verbose=True,
    #             ),
    #             {"C": [0.1, 0.01, 0.001, 1]},
    #             cv=5,
    #         ),
    #         "LogReg",
    #     )
    # )

    def do_split(split, X, Y, f, model, subjects):
        train, test = split
        train, test = subjects[train], subjects[test]
        train = Y["subject"].isin(train)
        test = Y["subject"].isin(test)
        Y_train, labels_dict = preprocess_label(Y[train], return_dict=True)
        # XXX: FIX ME
        # Y_test = preprocess_label(Y[test], use_dict=labels_dict)
        Y_test = preprocess_label(Y[test])
        X_train = np.array(X)[train.values]
        X_test = np.array(X)[test.values]
        clf = AugmentedClassifier(model, f)
        clf.fit(X_train, Y_train)
        score_split = clf.score(X_test, Y_test)
        return score_split

    scores = []
    for model, name in models:
        subjects = labels["subject"].unique()
        sf = ShuffleSplit(
            n_splits=n_splits, train_size=train_size, random_state=0
        )
        scores_split = Parallel(verbose=100, n_jobs=n_jobs)(
            delayed(do_split)(split, images_path, labels, f, model, subjects)
            for split in sf.split(range(len(subjects)))
        )
        for i_split, score_split in enumerate(scores_split):
            scores.append((method_name, name, score_split, i_split))

    scores = pd.DataFrame(
        scores, columns=["method_name", "algo", "score", "split"]
    )
    scores.to_csv(filename)

In [25]:
f = lambda X, Y: transform_based_augmentation(all_augs, mask, Z_inv, X, labels=Y, nb_fakes=200, n_jobs=-1, verbose=100)

In [26]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_size = 8

do_classif(
    data["images"],
    labels,
    f,
    method_name="all_augs",
    filename="results/hcp_all_augs.csv",
    train_size=train_size,
    n_splits=10,
    n_jobs=1,
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.


In [81]:
%debug

> <ipython-input-78-68a99804cc3d>(77)do_split()
     75         # Y_test = preprocess_label(Y[test], use_dict=labels_dict)
     76         Y_test = preprocess_label(Y[test])
---> 77         X_train = X[train]
     78         X_test = X[test]
     79         clf = AugmentedClassifier(model, f)

ipdb> type(X)
<class 'list'>
ipdb> train
0     True
1     True
2    False
3     True
4     True
5     True
6     True
7     True
8    False
9     True
Name: subject, dtype: bool
ipdb> X[train.values]
*** TypeError: only integer scalar arrays can be converted to a scalar index
ipdb> np.array(X)[train.values]
array(['/storage/store2/data/neurovault/collection_4337/image_84774.nii.gz',
       '/storage/store2/data/neurovault/collection_4337/image_86340.nii.gz',
       '/storage/store2/data/neurovault/collection_4337/image_72795.nii.gz',
       '/storage/store2/data/neurovault/collection_4337/image_78481.nii.gz',
       '/storage/store2/data/neurovault/collection_4337/image_76410.nii.gz',
       '/st